# Lab - Context Free Grammars and Parsing in the NLTK
# This file has small examples that are meant to be run individually in the Python shell

In [1]:
import nltk

## Part 1: create CFG and its parser

In [2]:
# write your own grammars
# include every words
grammar = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> Prop | Det N | Det N PP
  Prop -> "John" | "Mary" | "Bob" 
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)



In [3]:
# TOP-DOWN method: recursive descent parsing
# create the parser from a grammar
rd_parser = nltk.RecursiveDescentParser(grammar)


In [4]:
# test the parser with a simple sentence
# make an example sentence text
senttext = "Mary saw Bob"



In [5]:
# tokenize the sentence by splitting on white space
# use nltk.word_tokenize() for more complex example sentences
sentlist = senttext.split()
print(sentlist)

['Mary', 'saw', 'Bob']


In [6]:
# run the parse function on the list of tokens
trees = rd_parser.parse(sentlist)

In [7]:
# convert the generator to a list
treelist = list(trees)

In [8]:
# what type is an individual tree?
type(treelist[0])

nltk.tree.tree.Tree

In [9]:
# print the tree structures
for tree in treelist:
	print (tree)

(S (NP (Prop Mary)) (VP (V saw) (NP (Prop Bob))))


In [10]:
# try an ambiguous sentence
sent2list = "John saw the man in the park with a telescope".split()
for tree in rd_parser.parse(sent2list):
	print (tree)

(S
  (NP (Prop John))
  (VP
    (V saw)
    (NP
      (Det the)
      (N man)
      (PP
        (P in)
        (NP
          (Det the)
          (N park)
          (PP (P with) (NP (Det a) (N telescope))))))))
(S
  (NP (Prop John))
  (VP
    (V saw)
    (NP (Det the) (N man))
    (PP
      (P in)
      (NP
        (Det the)
        (N park)
        (PP (P with) (NP (Det a) (N telescope)))))))
(S
  (NP (Prop John))
  (VP
    (V saw)
    (NP (Det the) (N man) (PP (P in) (NP (Det the) (N park))))
    (PP (P with) (NP (Det a) (N telescope)))))


In [11]:
#  extend the grammar with more words (I, elephant, pajamas)
groucho_grammar = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked" | "shot"
  NP -> Prop | Det N | Det N PP
  Prop -> "John" | "Mary" | "Bob" | "I"
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park" | "elephant" | "pajamas"
  P -> "in" | "on" | "by" | "with"
  """)

In [12]:

# if we change the grammar, we create a new parser
rd_parser = nltk.RecursiveDescentParser(groucho_grammar)


In [13]:
# try sent4 with the recursive descent parser on groucho grammar
sent4list = "I shot an elephant in my pajamas".split()
for tree in rd_parser.parse(sent4list):
	print (tree)

(S
  (NP (Prop I))
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))
(S
  (NP (Prop I))
  (VP
    (V shot)
    (NP (Det an) (N elephant))
    (PP (P in) (NP (Det my) (N pajamas)))))


In [14]:
# extend the grammar for the flight grammar:  adding a rule to S and some words
flight_grammar = nltk.CFG.fromstring("""
  S -> NP VP | VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked" | "shot" | "book"
  NP -> Prop | Det N | Det N PP
  Prop -> "John" | "Mary" | "Bob" | "I"
  Det -> "a" | "an" | "the" | "my" | "that"
  N -> "man" | "dog" | "cat" | "telescope" | "park" | "elephant" | "pajamas" | "flight"
  P -> "in" | "on" | "by" | "with"
  """)

In [15]:
# make a recursive descent parser and parse the sentence
rd_parser = nltk.RecursiveDescentParser(flight_grammar)
sent5list = 'book that flight'.split()
for tree in rd_parser.parse(sent5list):
	print (tree)

(S (VP (V book) (NP (Det that) (N flight))))


## Part 2: Create PCFG and its parser

In [11]:
#   for transitive (TranV), intransitive (InV) and dative (DatV) verbs
prob_grammar = nltk.PCFG.fromstring("""
  S -> NP VP [0.9]| VP  [0.1]
  VP -> TranV NP [0.3]
  VP -> InV  [0.3]
  VP -> DatV NP PP  [0.4]
  PP -> P NP   [1.0]
  TranV -> "saw" [0.2] | "ate" [0.2] | "walked" [0.2] | "shot" [0.2] | "book" [0.2]
  InV -> "ate" [0.5] | "walked" [0.5]
  DatV -> "gave" [0.2] | "ate" [0.2] | "saw" [0.2] | "walked" [0.2] | "shot" [0.2]
  NP -> Prop [0.2]| Det N [0.4] | Det N PP [0.4]
  Prop -> "John" [0.25]| "Mary" [0.25] | "Bob" [0.25] | "I" [0.25] 
  Det -> "a" [0.2] | "an" [0.2] | "the" [0.2] | "my" [0.2] | "that" [0.2]
  N -> "man" [0.15] | "dog" [0.15] | "cat" [0.15] | "park" [0.15] | "telescope" [0.1] | "flight" [0.1] | "elephant" [0.1] | "pajamas" [0.1]
  P -> "in" [0.2] | "on" [0.2] | "by" [0.2] | "with" [0.2] | "through" [0.2]
  """)

In [17]:
# create a viterbi parser, a parser that expects a PCFG
viterbi_parser = nltk.ViterbiParser(prob_grammar)

In [18]:
# use its parse function on a list of tokens
for tree in viterbi_parser.parse(['John', 'saw', 'a', 'telescope']):
    print (tree)

(S
  (NP (Prop John))
  (VP (TranV saw) (NP (Det a) (N telescope)))) (p=2.16e-05)


In [19]:
# parse some other sentences
# this parser chooses to return the highest probability tree
for tree in viterbi_parser.parse(sent2list):
    print (tree)

(S
  (NP (Prop John))
  (VP
    (DatV saw)
    (NP
      (Det the)
      (N man)
      (PP (P in) (NP (Det the) (N park))))
    (PP (P with) (NP (Det a) (N telescope))))) (p=1.65888e-10)


In [20]:
for tree in viterbi_parser.parse(sent4list):
    print (tree)

(S
  (NP (Prop I))
  (VP
    (DatV shot)
    (NP (Det an) (N elephant))
    (PP (P in) (NP (Det my) (N pajamas))))) (p=4.608e-08)


## Part 3:  Dependency grammars

In [21]:
# a dependency grammar for the groucho example
# note difficulty of writing rules for every word dependency
groucho_dep_grammar = nltk.DependencyGrammar.fromstring("""
  'shot' -> 'I' | 'elephant' | 'in'
  'elephant' -> 'an' | 'in'
  'in' -> 'pajamas'
  'pajamas' -> 'my'
  """)
print (groucho_dep_grammar)

Dependency grammar with 7 productions
  'shot' -> 'I'
  'shot' -> 'elephant'
  'shot' -> 'in'
  'elephant' -> 'an'
  'elephant' -> 'in'
  'in' -> 'pajamas'
  'pajamas' -> 'my'


In [22]:
# create a dependency parser, assumes sentence is projective
pdp = nltk.ProjectiveDependencyParser(groucho_dep_grammar)
glist = 'I shot an elephant in my pajamas'.split()
# use the parse function to parse a list of tokens
trees = pdp.parse(glist)
for tree in trees:
    print (tree)

(shot I (elephant an (in (pajamas my))))
(shot I (elephant an) (in (pajamas my)))


## Week5 Lab

Starting with the flight grammar from the lab material (lab_cfgparsing.txt), add the CFG rules to parse the following four sentences. (Note that I have left the “.” off the end of each sentence.) 

I prefer a flight through Houston

Jack walked with the dog

John gave the dog a bone

I want to book that flight

First try using the rd_parser defined from the flight grammar to parse the sentence and then add words and/or add rules to the grammar. The first sentence is the easiest and the last sentence is probably the hardest.

When you try the first sentence, you will get an error that the grammar does not cover some of the input words, so you must add lexical rules that add the words to the appropriate categories.

But if there are not enough grammar rules, the rd_parser just quits with no parse trees, and you must add rules. You may also possibly want to add new types of rules. For example, in the last sentence, you should introduce a new type of phrase to handle “want to book”. There are a couple of difference ways to do this, but do not make “to book” be a prepositional phrase. In this case, it is probably better to treat the word “to” as a special case, instead of as a preposition.

Post your the revised grammar and the example parses for each sentence in your answer (screenshot of coding and output). 

In [52]:
flight_grammar = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP | V PP | V NP NP | V TO VP
  PP -> P NP 
  V -> "saw" | "ate"  | "walked"  | "book"  | "prefer" | "gave" | "want"
  NP -> Prop | Det N  | Det N PP  | N 
  Prop -> "John"  | "Mary"  | "Bob"  | "I" | "Jack"
  Det -> "a" | "an"  | "the"  | "my"  | "that" 
  N -> "man" | "dog"  | "flight"  | "Houston" | "bone"
  P -> "in"| "on" | "by"| "with" | "through" 
  TO ->  "to"
  """)

In [20]:
rd_parser = nltk.RecursiveDescentParser(flight_grammar)
sent6list = 'I prefer a flight through Houston'.split()
for tree in rd_parser.parse(sent6list):
	print (tree)

(S
  (NP (Prop I))
  (VP
    (V prefer)
    (NP (Det a) (N flight) (PP (P through) (NP (N Houston))))))
(S
  (NP (Prop I))
  (VP
    (V prefer)
    (NP (Det a) (N flight))
    (PP (P through) (NP (N Houston)))))


In [31]:
rd_parser = nltk.RecursiveDescentParser(flight_grammar)
sent7list = 'Jack walked with the dog'.split()
for tree in rd_parser.parse(sent7list):
	print (tree)

(S
  (NP (Prop Jack))
  (VP (V walked) (PP (P with) (NP (Det the) (N dog)))))


In [34]:
rd_parser = nltk.RecursiveDescentParser(flight_grammar)
sent8list = 'John gave the dog a bone'.split()
for tree in rd_parser.parse(sent8list):
	print (tree)

(S
  (NP (Prop John))
  (VP (V gave) (NP (Det the) (N dog)) (NP (Det a) (N bone))))


In [53]:
rd_parser = nltk.RecursiveDescentParser(flight_grammar)
sent9list = 'I want to book that flight'.split()
for tree in rd_parser.parse(sent9list):
	print (tree)

(S
  (NP (Prop I))
  (VP (V want) (TO to) (VP (V book) (NP (Det that) (N flight)))))
